# Testing the UF

## Downloading the 4th layer data from the soil moisture DB (Thingsboard)

In [1]:
# -*- coding: utf-8 -*-

#  downloading the soil moisture data using the thingsboard API

# Set up parameters for the simulation
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime as dt
import statistics as stats
import os
#*################ Global Parameters ################*#
# transmitter info
CTRS = ['0554','0556','0557','0558','0629','0630']  # Corn Transmitter IDs
CITRS = ['0630','0557']
CFTRS = ['0556','0629']
CRTRS = ['0554','0558']
STRS = ['0111','0112','0113','0114','0115','0116']   # Soybean Transmitter IDs
SITRS = ['0112','0114']
SFTRS = ['0113','0116']
SRTRS = ['0111','0115']

CED = pd.Timestamp(2024, 5, 1) # Corn Emergence Date
SED = pd.Timestamp(2024, 5, 15) # Soybean Emergence Date

CRZMD = 24 # Corn Root Zone Max Depth (in)
CRZMDW = 7 # Week of Corn reaches Root Zone Max Depth (at the end of the week)
CRZI = 4 # Corn Root Zone Initial Depth (in)
CRZIW = 1 # Week of Corn reaches Root Zone Initial Depth

SRZMD = 24 # Soybean Root Zone Max Depth (in)
SRZMDW = 7 # Week of Soybean reaches Root Zone Max Depth (at the end of the week)
SRZI = 4 # Soybean Root Zone Initial Depth (in)
SRZIW = 1 # Week of Soybean reaches Root Zone Initial Depth


#*################ Mendatory Parameters ################*#


def get_periods(CED, SED):
    # get today's date as string
    from datetime import date
    today = date.today().strftime("%Y%m%d")

    # Get today's date as datetime
    TDY = pd.Timestamp.today()
    TWK = TDY.isocalendar().week
    print('Today is :', today, '// Week:', TWK)
    # Corn Week From Emergence
    Cweek = CED.isocalendar().week
    CWPE = TWK - Cweek  # Corn Week From Emergence
    print('Emergence week of corn :', Cweek, '//',CWPE, 'weeks from now')

    # Soybean Week From Emergence
    Sweek = SED.isocalendar().week
    SWPE = TWK-Sweek  # Soybean Week From Emergence
    print('Emergence week of soybean :', Sweek, '//',SWPE, 'weeks from now')

    return today, TDY, CWPE, SWPE


def cal_rz(TDY, CDE, SDE, CRZI, SRZI, CRZMDW, SRZMDW, CRZMD, SRZMD):
    # this program is using linear group root growth rate
    # get the root zone depth for corn and soybean
    # get the days from emergence until the index date of dlydata (CDFE/SDFE)
    CDFE = (TDY - CED).days
    SDFE = (TDY - SED).days
    
    # root zone growth rate of corn (CRZGR/SRZGR) = CDFE*(CRZMD-CRZI)/(CRZMDW-1)*7 (inch/day)
    CRZGR = (CRZMD-CRZI)/((CRZMDW-1)*7)
    SRZGR = (SRZMD-SRZI)/((SRZMDW-1)*7)
    print('Root zone Growth rate for Corn:     ', round(CRZGR,4), ('inch/day'))
    print('Root zone Growth rate for Soybean:  ', round(SRZGR,4), ('inch/day'))

    # calculate the root zone depth for corn and soybean
    # if (CRZ/SRZ) is lower than CRZI/SRZI, then CRZ/SRZ is CRZI/SRZI
    # if (CRZ/SRZ) is higher than CRZMD/SRZMD, then CRZ/SRZ is equal to CRZMD, SRZMD
    
    CRZ = CRZI + CDFE*CRZGR
    CRZ = CRZMD if CRZ > CRZMD else CRZ
    SRZ = SRZI + SDFE*SRZGR
    SRZ = SRZMD if SRZ > SRZMD else SRZ
    

    # if CRZ/SRZ is below CRZI/SRZI, then CRZ/SRZ is 0
    # this is because the date is before the emergence date
    CRZ = round(CRZ if CRZ > CRZI else 0,4)
    SRZ = round(SRZ if SRZ > SRZI else 0,4)

    print('Root Zone Depth for Corn:    ', CRZ, ('inch'))    
    print('Root Zone Depth for Soybean: ', SRZ, ('inch'))

    return CRZ, SRZ

today, TDY, CWPE, SWPE = get_periods(CED, SED)
CRZ, SRZ = cal_rz(TDY, CWPE, SWPE, CRZI, SRZI, CRZMDW, SRZMDW, CRZMD, SRZMD)

Today is : 20240624 // Week: 26
Emergence week of corn : 18 // 8 weeks from now
Emergence week of soybean : 20 // 6 weeks from now
Root zone Growth rate for Corn:      0.4762 inch/day
Root zone Growth rate for Soybean:   0.4762 inch/day
Root Zone Depth for Corn:     24 inch
Root Zone Depth for Soybean:  23.0476 inch


In [2]:
# // this part is download the data from the website for 2 month.
import requests
import sys
import os
import json
import pandas as pd
from pprint import pprint
import datetime
import pytz
# import config  # I don't know what is this config but this is meaningless DK 2024-06-24
from dateutil.relativedelta import relativedelta

deviceList = []

# ** set the configuration for the request                                                                                  **
config = {
 'username' : 'yang2309@purdue.edu', ### Insert your email address used by AgIT Thingsboard system
 'password': 'dsya2002',  ### Insert your AgIT thingsboard password
 'server' : 'https://things.iot.ag.purdue.edu:8080'
}

# ** defining the function to get the token for the request and setting the header for the request                          **
def getCustomerDevices(custID, textSearch=None):
    parameters = {        
        'pageSize': 1000,
        'page': 0,                
    }
    att_parms = {
        'keys': 'dev_eui'
    }
    if(textSearch):
        parameters.update({'textSearch': textSearch})
    responseList = requests.get(f"{config['server']}/api/customer/{custID}/devices", headers=TBheaders,params= parameters).json()
    #pprint(responseList)
    list = []
    for dev in responseList['data']:
        #pprint(dev)
        #print('------------------------------------------------------------------------------------------')
        #'id': {'entityType': 'DEVICE', 'id': 'd49153a0-c868-11eb-95d8-09d06ef6a9a5'},
        url = f"{config['server']}/api/plugins/telemetry/DEVICE/{dev['id']['id']}/values/attributes"
        deviceResp = requests.get(url, headers=TBheaders,params= att_parms).json()
        #print('------------------------------------------------------------------------------------------')
        list.append([dev['id']['id'],dev['name'],deviceResp[0]['value']])
    return list
        

def login(url, username, password):
    # Log into ThingsBoard
    return requests.post(f"{url}/api/auth/login", json={
        "username": username,
        "password": password
    }).json()['token']

def get_keys(device):
    return requests.get(f"{config['server']}/api/plugins/telemetry/DEVICE/{device}/keys/timeseries",
                 headers=TBheaders).json()
def get_data_chunk(url, token, device, key, start, stop, limit):
    #print([url, device, key, start, stop, limit])
    return requests.get(f"{url}/api/plugins/telemetry/DEVICE/{device}/values/timeseries",
             headers=TBheaders,
            params= {
                'keys': key,
                'startTs': start,
                'endTs': stop,
                'limit': limit,
                'agg': 'NONE'
            }).json()

def get_data(url, token, device, key, start, stop):
    global totalLength
    p = pd.DataFrame()
    
    # You have to request data backwards in time ...
    while start < stop:
        data = get_data_chunk(url, token, device[0], key, start, stop, 100000)
        #print(data)
        if key not in data:
            break;
        
        #print(f"{key}: Loaded {len(data[key])} points")
        t = pd.DataFrame.from_records(data[key])
        #t['Timestamp'] = t['ts']
        #pprint(t['ts'])
        t['ts'] = (pd.to_datetime(t['ts'],unit='ms'))        
        t.set_index('ts', inplace=True)
        
        t.rename(columns={'value': key}, inplace=True)
        p = p._append(t)

        # Update "new" stop time
        stop = data[key][-1]['ts'] - 1
    totalLength += len(p)
    #print(f"Total Length: {totalLength}")
    return p

def outputCSV(devices):
    global totalLength
    final_df = pd.DataFrame()
    for device in devices:
        #print(f"Downloading DEVICE: {device[0]} data");
        #print(device)
        p = pd.DataFrame()
        for key in keys:
            #print(f"info: Pulling {key}...");
            tempin = get_data(config['server'], token, device, key, startTS, endTS)            
            if(len(tempin)>0):                
                p = pd.concat([p,tempin], axis=1)
        p['Entity Name'] = device[1]
        p['dev_eui'] = device[2]    
        p.reset_index(drop=False)
        #p_new_index = p.assign(**{'Timestamp': p.index})        
        if(len(p)):
            final_df = pd.concat([final_df,p])
        
    # Create Time Strings
    # Convert to nanoseconds for pandas.to_datetime
    start_timestamp_ns = startTS * 1000000
    end_timestamp_ns = endTS * 1000000
    
    # Convert timestamp to datetime object
    start_dt = pd.to_datetime(start_timestamp_ns, unit='ns')
    end_dt = pd.to_datetime(end_timestamp_ns, unit='ns')
    
    # Format datetime string as yyyy-mm-dd-HH-MM
    start_formatted_string = start_dt.strftime('%Y-%m-%d-%H-%M')
    end_formatted_string = end_dt.strftime('%Y-%m-%d-%H-%M')
    # Select variables to export
    df_order = ["Entity Name","data_soil_moisture4","dev_eui"]
    final_df = final_df.reindex(columns=df_order)
    final_df1 = final_df.sort_values(by='ts')
    # final_df1['Entity Name'] = final_df1['Entity Name'].str.replace('ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-','')
    
    # Get current time
    now = datetime.datetime.now()
    
    # Format time string (hours and minutes)
    formatted_time = now.strftime("%H-%M")
    # File saving directory
    final_df1.to_csv(f"./data-Layer4_{today}.csv")
    print("File Export Done.")

def getDeviceCredentialsByDeviceId(deviceID = 0):
    url = config['server']+'/device/'+deviceID+'/credentials'
    resp = requests.get(url,headers=TBheaders)
    responseList = resp.json()
    #pprint(responseList)
    return responseList['credentialsID']

def getDeviceServerAttributes(deviceID = 0):
    if deviceID == 0:
        while(deviceID == 0):
            try:
                deviceID = input("Enter device ID: ")
            except:
                print("Invalid DeviceID")
    url = config['server']+'/plugins/telemetry/DEVICE/'+deviceID+'/values/attributes'
    #pprint(url)
    #pprint(TBheaders)
    xresp = requests.get(url,headers=TBheaders)
    #pprint(xresp)
    #pprint(resp.content())
    #print(xresp.text())
    responseList = xresp.json()
    #pprint(responseList)
    #return responseList['credentialsID']


# ** getting token for the request                                                                                         **
print("Server: ",config['server'])
token = login(config['server'], config['username'], config['password']);
# print(f"Token: {token}")
TBheaders={ 'Accept': '*/*', 'X-Authorization': f"Bearer {token}" }



# Create a datetime object representing the local date and time
# Year, Month, Day, Hour, Minute
today_dt = datetime.datetime.now()
start = datetime.datetime.now()+ relativedelta(months=-2)

start_dt = datetime.datetime(min(CED, SED).year, min(CED, SED).month, min(CED, SED).day, 18, 0)
end_dt = datetime.datetime(today_dt.year, today_dt.month, today_dt.day, 6, 00)
print (start_dt, end_dt)

# Convert to a specific time zone (e.g., UTC)
start_tz_utc = pytz.timezone("UTC")
start_dt_utc = start_tz_utc.localize(start_dt)
end_tz_utc = pytz.timezone("UTC")
end_dt_utc = end_tz_utc.localize(end_dt)

# Extract the Unix timestamp
startTS = int(start_dt_utc.timestamp())*1000
endTS = int(end_dt_utc.timestamp())*1000

# Use for relative time frames
#startTS = int((datetime.now() - timedelta(days=30)  - datetime(1970, 1, 1)).total_seconds() * 1000) # 30 days ago
#endTS = int((datetime.datetime.utcnow() - datetime.datetime(1970, 1, 1)).total_seconds() * 1000) # now

# print(startTS, endTS)



# ** customer ID for the request                                                                                            **
# getCustomerDevices(custID, textSearch=None):
# 7576b020-ecae-11ec-b72b-5dd76ca52a2b = Cherkhauer Customer ID
# ABE-DRAGINO-GROPOINT-CHERKHAUER = Devices with names beginning with "ABE-DRAGINO-GROPOINT-CHERKHAUER"
devices = getCustomerDevices("7576b020-ecae-11ec-b72b-5dd76ca52a2b","ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE")
# pprint(devices)

totalLength = 0
# keys to retrieve
#keys = ["data_TempC_SHT","data_Hum_SHT"]
#keys = ["data_ambient_temperature","data_input1_frequency","data_input1_frequency_to_moisture","data_Input2_voltage","data_Input2_voltage_to_temp","data_light_intensity","data_relative_humidity"]
keys = ["data_soil_moisture4"]

outputCSV(devices)

Server:  https://things.iot.ag.purdue.edu:8080
2024-05-01 18:00:00 2024-06-24 06:00:00
File Export Done.


In [3]:
'''
organazing the downloaded data
    1. remove useless part from the station name
    2. organize as dataframe form
        2.1. frame into daily timesereis
        2.2. group into field types
    3. calculate the upperflux (UF) value following the equation

'''

# step 1. remove useless parts from the station name
def readraw_data(destination, filename, foutname):
    print ('File name is ::',filename)
    # open the file
    raw_data = pd.read_csv(filename,delimiter=',', parse_dates=['ts'],
                          dtype={'data_soil_moisture4':np.float64},
                          na_values=['Invalid data']
                          )
    raw_columns = raw_data.columns.tolist()            
    
    # change the name of the Entity Name column
    raw_data['Entity Name'] = raw_data['Entity Name'].str.replace('ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-','')
    stationlist = sorted(raw_data['Entity Name'].unique())

    # filter the data by stationlist and hour (12am - 5am)
    raw_data = raw_data[(raw_data['ts'].dt.hour >= 0) & (raw_data['ts'].dt.hour < 5)]
    L4df = raw_data.pivot(index='ts', columns='Entity Name', values=['data_soil_moisture4'])
    L4df.columns = L4df.columns.droplevel(0)
    stationlist = L4df.columns
            
    # find diff values and resample in 'D' for each stations
    # UFdf is empty dataframe for now
    UFdf = pd.DataFrame(index=L4df.index.copy())
    UFdf = UFdf.resample('D').sum()
    for i in stationlist:
        tempdf = L4df[[i]]        
        tempdf = tempdf.dropna()
        # get the hourly UF
        tempdf = tempdf.diff()
        # aggregate into daily
        tempdf = tempdf.resample('D').sum()
        UFdf = pd.concat([UFdf, tempdf], axis=1)
    
    return UFdf

In [16]:
def UF_cal(UFdf):
    print(UFdf.columns.to_list())
    # calculating mean values for each managements
    # list of transmitters below
    # make average to get UF for each managements
    list_TRS = [CITRS, CFTRS, CRTRS, SITRS, SFTRS, SRTRS]
    UFdf['CITRS'] = UFdf[CITRS].mean(axis=1)
    UFdf['CFTRS'] = UFdf[CFTRS].mean(axis=1)
    UFdf['CRTRS'] = UFdf[CRTRS].mean(axis=1)

    UFdf['SITRS'] = UFdf[SITRS].mean(axis=1)
    UFdf['SFTRS'] = UFdf[SFTRS].mean(axis=1)
    UFdf['SRTRS'] = UFdf[SRTRS].mean(axis=1)

    # extract UF for each plot managements
    UFdf2 = UFdf[['CITRS', 'CFTRS', 'CRTRS', 'SITRS', 'SFTRS', 'SRTRS']]
    # Unit conversion from % to inch. (6 = layer depth)
    UFdf2 = UFdf * 0.01 * 6
    UFdf2 = UFdf.round(decimals=4)
    
    return UFdf, UFdf2

In [17]:
'''
IMPLIMENTATION BLOCK
'''

destination = './'
filename = f"./data-Layer4_{today}.csv"
foutname = f"./CData_L4_{today}.csv"

UFdf = readraw_data(destination, filename, foutname)
UFdf, UFdf2 = UF_cal(UFdf)

File name is :: ./data-Layer4_20240624.csv
['0111', '0112', '0113', '0114', '0115', '0116', '0554', '0556', '0557', '0558', '0629', '0630']


In [18]:

"""
positive value means there was increase of the moisture on that day compare to the previous day
"""
UFdf2

,0111,0112,0113,0114,0115,0116,0554,0556,0557,0558,0629,0630,CITRS,CFTRS,CRTRS,SITRS,SFTRS,SRTRS
ts,,,,,,,,,,,,,,,,,,
2024-05-02,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,NaN,NaN
2024-05-03,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.20,NaN,NaN
2024-05-04,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.20,NaN,NaN
2024-05-05,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,NaN,NaN
2024-05-06,NaN,-0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.30,NaN,NaN
2024-05-07,NaN,-0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.30,NaN,NaN
2024-05-08,NaN,-0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.20,NaN,NaN
2024-05-09,NaN,-0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.10,NaN,NaN
2024-05-10,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN
